In [1]:
!pip install -U accelerate bitsandbytes datasets peft transformers tokenizers trl
!pip install -U googletrans==3.1.0a0 langdetect

  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/2c/92/48aec3736ca778ffe5fa68e19e3c18917cba4de43fa46fe6176cccafe267/accelerate-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for bitsandbytes from https://files.pythonhosted.org/packages/e4/e6/ccb84da7ffaf208a71c2c3c8e1120b34759df640db959660be9a98505eb4/bitsandbytes-0.44.1-py3-none-manylinux_2_24_x86_64.whl.metadata
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/be/3e/e58d4db4cfe71e3ed07d169af24db30cfd582e16f977378bd43fd7ec1998/datasets-3.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/78/9d/5f95bfb298c8d3b4e3a107701f9a4e7774a0d4d1f8eb0c9d5420b80f7c9d/peft-0.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/f9/9d/030cc1b3e88172967e22ee1d012e0d5e0384eb70d2a098d1669d549aea29/transfor

## IMPORT

In [2]:
import os
import sys
import transformers
import torch
import numpy as np
import pandas as pd
import gc
from trl import SFTTrainer
import trl
from nltk.stem import WordNetLemmatizer
import spacy
import tqdm
# import timms
import unicodedata
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## JUST TO BE SAFE IMPORTS

In [3]:
import googletrans
from langdetect import detect

In [4]:
import googletrans
translator = googletrans.Translator()
googletrans.__version__

'3.1.0-alpha'

In [5]:
from langdetect import detect_langs


In [6]:
import warnings
warnings.filterwarnings("ignore")

## CONFIG

In [7]:
CONFIG={
    "device" : torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    "data" : "/kaggle/input/6000-indian-food-recipes-dataset/IndianFoodDatasetCSV.csv",
    "modelhf" : "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1",
    "modelno" : "/kaggle/input/mistral/pytorch/7b-v0.1/1",
    "model" : "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1",
    "trained model" : " ",
    "batch size" : 4,
    "format for training" : "chat ml",
}

## UTILITY

In [8]:
con = CONFIG
device = con['device']

In [9]:
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

In [10]:
features=["RecipeName","Servings","Ingredients","TotalTimeInMins","Instructions"]

In [11]:
def detect_translate(text):
    language = detect_langs(text)
    if "hi" in str(language):
        translated = translator.translate(text).text
        return translated
    return text

In [12]:
train_text = """
Recipe name:{} serves {}

Ingredients needed include:
{}


Time required: {}

Steps are as follows:
{}

"""

In [13]:
def clean_text_instruct(text):
    text = unicodedata.normalize("NFKD",str(text)) # sub /xa0 with space 
    text = text.replace(".", ".\n")
    return text

def clean_text_ingredients(text):
    text = unicodedata.normalize("NFKD",str(text)) # sub /xa0 with space 
    text = text.replace(",", ".\n")
    return text

In [14]:
system_prompt="This is a chat between a user and an assistant. The following instructions in this system prompt are all directed at the assistant. Assistant, you are a chef and you give recipe instructions to curious users. As an assistant, prioritize helpful, respectful, and honest responses. When a user asks for a recipe, give them clear, detailed, step-by-step instructions on how to make that recipe from scratch in one message. Give the user a detailed explanation for every instruction that the recipe requires. Avoid harmful or inappropriate content. If a question is unclear, explain why. Refrain from sharing false information when unsure."

In [15]:
message_chatml=[
    {"role":"system","content": system_prompt},
    {"role":"user","content":"Hi, can you please give me a suggestion for a recipe I can cook with {}"},
    {"role":"assistant","content":"Certainly! How about trying a classic and delicious recipe {}"}
               ]

In [16]:
gc.collect()

30

## PREPROCESSING DATA

In [19]:
data = pd.read_csv("/kaggle/input/manually-labeled-dataset/fixedRecipesDataset.csv")
data.head()

,Srno,RecipeName,Ingredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Course,Diet,Instructions,IngredientFeature
0,1,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...",karela
1,2,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato...",rice tomato
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...",vermicelli
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,chicken gongura
4,5,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ...",ginger


In [21]:
try:
    drop_features = ["Ingredients","Instructions","RecipeName","Cuisine","URL"]
    data=data.drop(columns=drop_features)
    rename_features=["Translated"+i for i in drop_features[:3]]
    df=drop_features
    rf=rename_features
    data=data.rename(columns={rf[0]:df[0],rf[1]:df[1],rf[2]:df[2]})
    del df,drop_features,rename_features
except:
    pass
data.head()

,Srno,RecipeName,Ingredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Course,Diet,Instructions,IngredientFeature
0,1,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...",karela
1,2,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato...",rice tomato
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...",vermicelli
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,chicken gongura
4,5,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ...",ginger


In [22]:
def fill_text(index,text = train_text,features = features, data = data):
    df = data[features].iloc[index]
    text = text.format(df.RecipeName,
                       df.Servings,
                       clean_text_ingredients(df.Ingredients),
                       df.TotalTimeInMins,
                       clean_text_instruct(df.Instructions))
    return text

In [23]:
def create_dataset(data,message,tokenizer):
    dataset = []
    for i in tqdm.tqdm(range(data.shape[0])):
        input_ = message
        ingredients =data.IngredientFeature.iloc[i]
        recipe =fill_text(i)
        input_[1]["content"] = input_[1]["content"].format(ingredients) #users
        input_[2]["content"] = input_[2]["content"].format(recipe) #assistant
        tokenized = tokenizer.apply_chat_template(input_, return_tensors="pt")
        converted = tokenizer.batch_decode(tokenized)
        dataset.append({"messages":converted[0]})
    dataset
    return dataset        

# example

In [24]:
"""
Recipe name:Masala Karela Recipe serves 6

Ingredients needed include:6 Karela (Bitter Gourd/ Pavakkai) - deseeded,
Salt - to taste,
1 Onion - thinly sliced,
3 tablespoon Gram flour (besan),
2 teaspoons Turmeric powder (Haldi),
1 tablespoon Red Chilli powder,
2 teaspoons Cumin seeds (Jeera),
1 tablespoon Coriander Powder (Dhania),
1 tablespoon Amchur (Dry Mango Powder),
Sunflower Oil - as required

Time required: 45 mins

Steps are as follows:

To begin making the Masala Karela Recipe,de-seed the karela and slice.
Do not remove the skin as the skin has all the nutrients.
Add the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles.
Release the pressure immediately and open the lids. 
Keep aside.Heat oil in a heavy bottomed pan or a kadhai. 
Add cumin seeds and let it sizzle.
Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.
Add the karela, red chilli powder, amchur powder, coriander powder and besan.
Stir to combine the masalas into the karela.
Drizzle a little extra oil on the top and mix again.
Cover the pan and simmer Masala Karela stirring occasionally until everything comes together well.
Turn off the heat.Transfer Masala Karela into a serving bowl and serve.
Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family. 
"""

'\nRecipe name:Masala Karela Recipe serves 6\n\nIngredients needed include:6 Karela (Bitter Gourd/ Pavakkai) - deseeded,\nSalt - to taste,\n1 Onion - thinly sliced,\n3 tablespoon Gram flour (besan),\n2 teaspoons Turmeric powder (Haldi),\n1 tablespoon Red Chilli powder,\n2 teaspoons Cumin seeds (Jeera),\n1 tablespoon Coriander Powder (Dhania),\n1 tablespoon Amchur (Dry Mango Powder),\nSunflower Oil - as required\n\nTime required: 45 mins\n\nSteps are as follows:\n\nTo begin making the Masala Karela Recipe,de-seed the karela and slice.\nDo not remove the skin as the skin has all the nutrients.\nAdd the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles.\nRelease the pressure immediately and open the lids. \nKeep aside.Heat oil in a heavy bottomed pan or a kadhai. \nAdd cumin seeds and let it sizzle.\nOnce the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.\nAdd the karela, red ch

In [25]:
data.columns

Index(['Srno', 'RecipeName', 'Ingredients', 'PrepTimeInMins', 'CookTimeInMins',
       'TotalTimeInMins', 'Servings', 'Course', 'Diet', 'Instructions',
       'IngredientFeature'],
      dtype='object')

In [26]:
fill_text(0) # example

'\nRecipe name:Masala Karela Recipe serves 6\n\nIngredients needed include:\n6 Karela (Bitter Gourd/ Pavakkai) - deseeded.\nSalt - to taste.\n1 Onion - thinly sliced.\n3 tablespoon Gram flour (besan).\n2 teaspoons Turmeric powder (Haldi).\n1 tablespoon Red Chilli powder.\n2 teaspoons Cumin seeds (Jeera).\n1 tablespoon Coriander Powder (Dhania).\n1 tablespoon Amchur (Dry Mango Powder).\nSunflower Oil - as required\n\n\nTime required: 45\n\nSteps are as follows:\nTo begin making the Masala Karela Recipe,de-seed the karela and slice.\n Do not remove the skin as the skin has all the nutrients.\n Add the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles.\n Release the pressure immediately and open the lids.\n Keep aside.\nHeat oil in a heavy bottomed pan or a kadhai.\n Add cumin seeds and let it sizzle.\nOnce the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.\nAdd the karela, red 

In [27]:
length = data.shape[0]

In [28]:
ing_features = ['Ingredients','RecipeName']
"""
Finding common words in recipename and ingredients 
"""
# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')

def lem(text):
    doc = nlp(text)
    lemmatized = [token.lemma_ for token in doc]
    return lemmatized 

def ing_for_prompt(data):
    name_=data.RecipeName
    ing_=data.Ingredients
    column_common_words = []
    for i in tqdm.tqdm(range(length)):
        a1=str(name_.iloc[i]).lower().split(",")
        a2=str(ing_.iloc[i]).lower().split(",")
        a1=lem(" ".join(a1))
        a2=lem(" ".join(a2))
        if len(a2)>len(a1):
            a1, a2 = a2, a1
        common_words = []
        for j in a1:
            for k in a2:
                if j==k and len(j)>1:
                    common_words.append(j)
                    break
        column_common_words.append(" ".join(common_words))
    data["IngredientFeature"] = column_common_words
    return data

data=ing_for_prompt(data)
data

100%|██████████| 6871/6871 [02:26<00:00, 46.75it/s]


,Srno,RecipeName,Ingredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Course,Diet,Instructions,IngredientFeature
0,1,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...",karela
1,2,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato...",rice tomato
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...",vermicelli
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,chicken gongura
4,5,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ...",
...,...,...,...,...,...,...,...,...,...,...,...
6866,14073,Goan Mushroom Xacuti Recipe,"20 बटन मशरुम,2 प्याज - काट ले,1 टमाटर - बारीक ...",15,45,60,4,Lunch,Vegetarian,गोअन मशरुम जकुटी रेसिपी बनाने के लिए सबसे पहले...,
6867,14107,Sweet Potato & Methi Stuffed Paratha Recipe,"1 बड़ा चम्मच तेल,1 कप गेहूं का आटा,नमक - स्वाद ...",30,60,90,4,North Indian Breakfast,Diabetic Friendly,शकरकंदी और मेथी का पराठा रेसिपी बनाने के लिए स...,
6868,14165,Ullikadala Pulusu Recipe | Spring Onion Curry,150 grams Spring Onion (Bulb & Greens) - chopp...,5,10,15,2,Side Dish,Vegetarian,To begin making Ullikadala Pulusu Recipe | Spr...,spring onion
6869,14167,Kashmiri Style Kokur Yakhni Recipe-Chicken Coo...,"1 kg Chicken - medium pieces,1/2 cup Mustard o...",30,45,75,4,Lunch,Non Vegeterian,To begin making the Kashmiri Kokur Yakhni reci...,chicken yogurt


In [29]:
# for i in data.IngredientFeature.iloc[3:5]:
#     print(i)
#     print()
count=0
ilocs=[]
for i in range(length):
    text=data["IngredientFeature"].iloc[i]
    if text=="":
        count+=1
        ilocs.append(i)
print(count,"have to manually written")

1457 have to manually written


In [30]:
def manual_entry(data,ilocs=ilocs):
    name_=data.RecipeName
    ing_=data.Ingredients
    words_=data.IngredientFeature
    for i in ilocs:
        print(name_.iloc[i],"\n",ing_.iloc[i],"\n",words_.iloc[i],"\n")
        words_.iloc[i]=input("give data->")
    data["IngredientFeature"]=words_
    return data
# data=manual_entry(data)
# # data added manually by DWOK
data_m = pd.read_csv("/kaggle/input/manually-labeled-dataset/fixedRecipesDataset.csv")
data_m.head()

,Srno,RecipeName,Ingredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Course,Diet,Instructions,IngredientFeature
0,1,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...",karela
1,2,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato...",rice tomato
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...",vermicelli
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,chicken gongura
4,5,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ...",ginger


In [31]:
data_m.shape

(6871, 11)

In [32]:
def extract_ingredients(series):
    ingredient=[]
    for i in tqdm.tqdm(series):
        ingredient+=i.split()
    ingredient=list(set(ingredient))
    return ingredient
        
LIST_OF_INGREDIENTS = extract_ingredients(data_m.IngredientFeature.iloc[:])

100%|██████████| 6871/6871 [00:00<00:00, 971516.41it/s]


In [33]:
len(LIST_OF_INGREDIENTS)
# run till here dwok

1003

In [34]:
# 
ing_ = data_m.Ingredients
words_ = data_m.Instructions
for i in tqdm.tqdm(range(data_m.shape[0])):
    
    ing_.iloc[i] = detect_translate(str(ing_.iloc[i]))
    words_.iloc[i] = detect_translate(str(words_.iloc[i]))
    
data_m.Ingredients = ing_
data_m.Instructions = words_
data_m

100%|██████████| 6871/6871 [04:48<00:00, 23.80it/s]


,Srno,RecipeName,Ingredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Course,Diet,Instructions,IngredientFeature
0,1,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...",karela
1,2,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato...",rice tomato
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...",vermicelli
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,chicken gongura
4,5,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ...",ginger
...,...,...,...,...,...,...,...,...,...,...,...
6866,14073,Goan Mushroom Xacuti Recipe,"20 button mushrooms, 2 onions - chopped, 1 tom...",15,45,60,4,Lunch,Vegetarian,"To make Goan Mushroom Jacuti recipe, first was...",mushroom
6867,14107,Sweet Potato & Methi Stuffed Paratha Recipe,"1 tbsp oil, 1 cup wheat flour, salt - as per t...",30,60,90,4,North Indian Breakfast,Diabetic Friendly,To make Sweet Potato and Fenugreek Paratha Rec...,sweet potato
6868,14165,Ullikadala Pulusu Recipe | Spring Onion Curry,150 grams Spring Onion (Bulb & Greens) - chopp...,5,10,15,2,Side Dish,Vegetarian,To begin making Ullikadala Pulusu Recipe | Spr...,spring onion
6869,14167,Kashmiri Style Kokur Yakhni Recipe-Chicken Coo...,"1 kg Chicken - medium pieces,1/2 cup Mustard o...",30,45,75,4,Lunch,Non Vegeterian,To begin making the Kashmiri Kokur Yakhni reci...,chicken yogurt


In [35]:
data_m.IngredientFeature.dtype

dtype('O')

## TRAINING

In [36]:
model = AutoModelForCausalLM.from_pretrained(con["model"],
#                                              load_in_4bit=True,
                                             quantization_config=bnb_config,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                             trust_remote_code=True
                                            )
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [37]:
tokenizer = AutoTokenizer.from_pretrained(con["model"], trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

## TESTING

In [38]:
messages = [
    {"role": "system", "content" : system_prompt},
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have brownies recipes?"}
]

In [51]:
tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}"
encodeds = tokenizer.apply_chat_template(messages, tokenize=True ,return_tensors="pt")

model_inputs = encodeds.to(device)
print(model_inputs)
# model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


tensor([[  523, 28766,   321, 28730,  2521, 28766, 28767,  6574,    13,  3260,
           349,   264, 10706,  1444,   264,  2188,   304,   396, 13892, 28723,
           415,  2296, 11382,   297,   456,  1587, 11510,   460,   544, 10666,
           438,   272, 13892, 28723, 21631, 28725,   368,   460,   264, 16749,
           304,   368,  2111, 13405, 11382,   298, 13903,  5443, 28723,  1136,
           396, 13892, 28725,  4681, 23175, 10865, 28725,  3116,  1007, 28725,
           304,  6858, 14915, 28723,  1684,   264,  2188, 12373,   354,   264,
         13405, 28725,  2111,   706,  3081, 28725, 10537, 28725,  3707, 28733,
          1403, 28733,  7691, 11382,   356,   910,   298,  1038,   369, 13405,
           477, 15147,   297,   624,  2928, 28723, 16104,   272,  2188,   264,
         10537, 13268,   354,  1012, 13126,   369,   272, 13405,  6948, 28723,
         27038, 26299,   442,   297, 25381,   380,  3036, 28723,  1047,   264,
          2996,   349,   521,  6206, 28725,  7282,  

In [52]:
model,tokenizer=trl.setup_chat_format(model, tokenizer)

In [53]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"]
)
model = get_peft_model(model, peft_config)

In [54]:
data_m.head()

,Srno,RecipeName,Ingredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Course,Diet,Instructions,IngredientFeature
0,1,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...",karela
1,2,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato...",rice tomato
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...",vermicelli
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,chicken gongura
4,5,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ...",ginger


In [55]:
dataset=create_dataset(data_m.iloc[:int(length*(0.55))],message_chatml,tokenizer)
print(dataset[0]["messages"])

100%|██████████| 3779/3779 [00:10<00:00, 350.07it/s]

<|im_start|> system
This is a chat between a user and an assistant. The following instructions in this system prompt are all directed at the assistant. Assistant, you are a chef and you give recipe instructions to curious users. As an assistant, prioritize helpful, respectful, and honest responses. When a user asks for a recipe, give them clear, detailed, step-by-step instructions on how to make that recipe from scratch in one message. Give the user a detailed explanation for every instruction that the recipe requires. Avoid harmful or inappropriate content. If a question is unclear, explain why. Refrain from sharing false information when unsure.<|im_end|> 
<|im_start|> user
Hi, can you please give me a suggestion for a recipe I can cook with karela<|im_end|> 
<|im_start|> assistant
Certainly! How about trying a classic and delicious recipe 
Recipe name:Masala Karela Recipe serves 6

Ingredients needed include:
6 Karela (Bitter Gourd/ Pavakkai) - deseeded.
Salt - to taste.
1 Onion - t

In [56]:
training_arguments = TrainingArguments(
    output_dir="/kaggle/working/",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= 500,
    dataset_text_field="messages",
#     formatting_func=formatting_prompts_func,
    tokenizer=tokenizer,
    args=training_arguments,
    packing= True,
)

Generating train split: 0 examples [00:00, ? examples/s]

In [57]:
import time 

In [60]:
# help(trainer.model.save_pretrained("/kaggle/working/"))

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      True if self else False
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [58]:
try:
    try:
        trainer.train()
    except:
        trainer.model.save_pretrained("/kaggle/working/")
except:
    print("saving the best model",trainer.model)

Step,Training Loss


NameError: name 'new_model' is not defined

## EVALUATION

In [61]:
model=trainer.model

In [62]:
messages = [
    {"role": "system", "content" : system_prompt},
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have brownies recipes?"}
]
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj

In [63]:
encodeds = tokenizer.apply_chat_template(messages, tokenize=True ,return_tensors="pt")

model_inputs = encodeds.to(device)
print(model_inputs)
# model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

tensor([[32000,  1587,    13,  3260,   349,   264, 10706,  1444,   264,  2188,
           304,   396, 13892, 28723,   415,  2296, 11382,   297,   456,  1587,
         11510,   460,   544, 10666,   438,   272, 13892, 28723, 21631, 28725,
           368,   460,   264, 16749,   304,   368,  2111, 13405, 11382,   298,
         13903,  5443, 28723,  1136,   396, 13892, 28725,  4681, 23175, 10865,
         28725,  3116,  1007, 28725,   304,  6858, 14915, 28723,  1684,   264,
          2188, 12373,   354,   264, 13405, 28725,  2111,   706,  3081, 28725,
         10537, 28725,  3707, 28733,  1403, 28733,  7691, 11382,   356,   910,
           298,  1038,   369, 13405,   477, 15147,   297,   624,  2928, 28723,
         16104,   272,  2188,   264, 10537, 13268,   354,  1012, 13126,   369,
           272, 13405,  6948, 28723, 27038, 26299,   442,   297, 25381,   380,
          3036, 28723,  1047,   264,  2996,   349,   521,  6206, 28725,  7282,
          2079, 28723,  6360, 28712,   426,   477, 1